In [1]:
# TODO: Special treatment to 0 and 1 in formula.
# TODO: Compare terms to ensure each term appears only once in ModelFormula

In [2]:
import formulae as fm
import pandas as pd
import numpy as np

def parse(code):
    return fm.Parser(fm.Scanner(code).scan_tokens()).parse()

In [3]:
parse("y ~ x + z")

Binary(left=Variable(y), op=~, right=Binary(left=Variable(x), op=+, right=Variable(z)))

In [4]:
data = pd.DataFrame({
    'x': np.random.normal(size=10),
    'y': np.random.normal(size=10),
    'z': np.random.normal(size=10)
})
fm.eval_in_data_mask('x / y', data=data, n=1)

0    6.279384
1   -0.280927
2   -0.465813
3    1.096737
4   -3.723105
5    4.418416
6   -0.483899
7    0.103572
8   -0.434913
9    6.634665
dtype: float64

In [5]:
fm.Resolver(parse("x")).resolve()

Term(
  name= x
  variable= x
  kind= None
  data= None
)

In [6]:
fm.Resolver(parse("x + y")).resolve()

ModelTerms(
  Term(
    name= x
    variable= x
    kind= None
    data= None
  ),
  Term(
    name= y
    variable= y
    kind= None
    data= None
  )
)

In [7]:
fm.Resolver(parse("x + z - z")).resolve()

ModelTerms(
  Term(
    name= x
    variable= x
    kind= None
    data= None
  )
)

In [8]:
fm.Resolver(parse("x + z - y + m")).resolve()

ModelTerms(
  Term(
    name= m
    variable= m
    kind= None
    data= None
  ),
  Term(
    name= x
    variable= x
    kind= None
    data= None
  ),
  Term(
    name= z
    variable= z
    kind= None
    data= None
  )
)

In [9]:
fm.Resolver(parse("y ~ a + b")).resolve()

ModelTerms(
  ResponseTerm(
    name= y
    variable= y
    kind= None
    data= None
  ),
  Term(
    name= a
    variable= a
    kind= None
    data= None
  ),
  Term(
    name= b
    variable= b
    kind= None
    data= None
  )
)

In [10]:
fm.Resolver(parse("y ~ a + b + a:b + c + a:b + a:b:c")).resolve()

ModelTerms(
  ResponseTerm(
    name= y
    variable= y
    kind= None
    data= None
  ),
  Term(
    name= b
    variable= b
    kind= None
    data= None
  ),
  Term(
    name= a
    variable= a
    kind= None
    data= None
  ),
  Term(
    name= c
    variable= c
    kind= None
    data= None
  ),
  InteractionTerm(
    name= a:b
    variables= {'b', 'a'}
  ),
  InteractionTerm(
    name= a:b:c
    variables= {'b', 'a', 'c'}
  )
)